In [1]:
import numpy as np
import random
import math
import random
import telebot

In [2]:
def randomGeneration():
  return np.random.permutation(100)

In [3]:
def read_city_data(filename):
    city_data = []
    with open(filename, 'r') as f:
        for line in f:
            city, lat, lon = line.strip().split(' ')
            city_data.append([city, int(lat), int(lon)])
            # check city is good verification 
            # id , lat (x), long(y) in array
    return city_data

In [4]:
city_data = read_city_data('city-cor.txt')
print(len(city_data))
# for city, lat, lon in city_data:
#     print(f"City: {city}, Latitude: {lat}, Longitude: {lon}")

100


In [5]:
def read_city_answer(filename):
    city_answer = []
    with open(filename, 'r') as f:
        for line in f:
            city_index = int(line)
            city_answer.append(city_index)
            # id , lat (x), long(y) in tupple
    return city_answer

In [6]:
city_ans = read_city_answer('answer.txt')
city_ans

[1,
 47,
 93,
 28,
 67,
 58,
 61,
 51,
 87,
 25,
 81,
 69,
 64,
 40,
 54,
 2,
 44,
 50,
 73,
 68,
 85,
 82,
 95,
 13,
 76,
 33,
 37,
 5,
 52,
 78,
 96,
 39,
 30,
 48,
 100,
 41,
 71,
 14,
 3,
 43,
 46,
 29,
 34,
 83,
 55,
 7,
 9,
 57,
 20,
 12,
 27,
 86,
 35,
 62,
 60,
 77,
 23,
 98,
 91,
 45,
 32,
 11,
 15,
 17,
 59,
 74,
 21,
 72,
 10,
 84,
 36,
 99,
 38,
 24,
 18,
 79,
 53,
 88,
 16,
 94,
 22,
 70,
 66,
 26,
 65,
 4,
 97,
 56,
 80,
 31,
 89,
 42,
 8,
 92,
 75,
 19,
 90,
 49,
 6,
 63]

In [7]:
print(city_data[0])

['1', 1380, 939]


In [8]:
def generate_random_chromosomes(population_size):
    chromosomes = []
    for _ in range(population_size):
        chromosome = randomGeneration()
        # random.shuffle(chromosome)
        chromosomes.append(chromosome)
    return chromosomes

In [9]:
def calculate_fitness_np(chromosome, cities=city_data):
    if len(chromosome) != 100 :
        print('wrong chromosome length')
        raise ValueError

    total_distance = 0
    for i in range(100):
        city1_index = chromosome[i-1]
        city2_index = chromosome[i]
        
        city1 = np.array(cities[city1_index-1][1:])
        city2 = np.array(cities[city2_index-1][1:])
        
        distance = np.linalg.norm(city1 - city2)
        total_distance += distance

    return int(total_distance // 1)

In [10]:
calculate_fitness_np(city_ans)

21285

In [11]:
population = generate_random_chromosomes(500)
print(population[0])

[ 9 92 80  1 61 19 24 17 58 62 27 42 67 89 48 29 14 49 95 90 25  6 22 99
 69 91 50 44 39 53 15 87 81 16 82 28 77 79 65 78  5 74  7 54 64 40 93 12
 63 84 33 70 20 56 51 21 30 31 47 52 96  3 71 38 57 23 86 72 60 32  0 75
 45  8 35 43 37 83 88 55 34 41 66 26 73 85 36 59  4 11 76 97  2 46 13 94
 98 68 18 10]


In [13]:
fitness_values = np.array([calculate_fitness_np(chromosome, cities=city_data) for chromosome in population])

In [15]:
def permutation_parent_selection(population, fitness_values, selection_method="roulette_wheel"):
  """Selects parents for a genetic algorithm using permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.
    selection_method: The parent selection method to use. Supported methods are
      "roulette_wheel" and "tournament".

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  if selection_method == "roulette_wheel":
    return roulette_wheel_permutation_parent_selection(population, fitness_values)
  elif selection_method == "tournament":
    return tournament_permutation_parent_selection(population, fitness_values)
  else:
    raise ValueError("Unsupported parent selection method: {}".format(selection_method))


def roulette_wheel_permutation_parent_selection(population, fitness_values):
  """Selects parents using roulette wheel selection for permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  total_fitness = np.sum(fitness_values)
  fitness_probabilities = fitness_values / total_fitness
  cumulative_probabilities = np.cumsum(fitness_probabilities)

  random_value = random.random()
  parent1_index = 0
  for i, probability in enumerate(cumulative_probabilities):
    if random_value < probability:
      parent1 = population[i]
      break

  random_value = random.random()
  parent2_index = 0
  for i, probability in enumerate(cumulative_probabilities):
    if random_value < probability:
      parent2 = population[i]
      break

  return parent1, parent2


def tournament_permutation_parent_selection(population, fitness_values, tournament_size=2):
  """Selects parents using tournament selection for permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.
    tournament_size: The size of the tournament.

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  parent1 = None
  parent2 = None

  for _ in range(tournament_size):
    random_index = random.randint(0, len(population) - 1)

    if parent1 is None or fitness_values[random_index] > fitness_values[parent1]:
      parent1 = population[random_index]

    if parent2 is None or fitness_values[random_index] > fitness_values[parent2]:
      parent2 = population[random_index]

  return parent1, parent2


In [16]:
roulette_wheel_permutation_parent_selection(population=population, fitness_values=fitness_values)

(array([34, 75, 56,  8, 39, 93, 31, 52, 61, 40, 47,  2, 85, 87,  7, 11, 32,
        10, 70, 90, 80, 59, 26, 54, 81, 76, 71, 97, 60, 17, 83, 74, 49, 96,
        92, 64, 19, 82, 95, 77, 44, 58, 14, 41, 25, 63,  4, 51, 98, 29, 12,
        94, 21, 43,  5, 24, 37, 48, 86, 78, 89, 99,  1, 16, 68, 42, 33, 22,
        62, 13, 20,  9,  0, 79, 65, 35, 69, 55, 18, 72,  6,  3, 38, 30, 23,
        88, 73, 66, 45, 15, 28, 50, 53, 91, 67, 46, 57, 27, 36, 84]),
 array([88, 98, 37, 42, 18, 85, 59,  9, 60, 84, 80, 13, 16, 99, 36, 44, 66,
        62, 79, 14, 24, 12, 49, 31, 75, 96, 38, 76, 11, 74, 55, 89, 35,  8,
        68, 86,  2, 95, 90, 17,  1,  0, 20, 64, 67, 23,  4, 28, 41, 54,  7,
        32, 40, 21, 58, 29, 30, 69, 65, 71, 78, 87, 56, 46, 22, 47, 82, 19,
        27, 63, 26, 73, 51, 72, 61, 94, 52, 34, 93,  3, 10,  5, 39, 92, 77,
        15, 25, 97, 83, 70, 45, 81, 33,  6, 43, 91, 50, 57, 48, 53]))

In [17]:
def PMX_crossover(parent1, parent2, seed):
    # key = random.PRNGKey(72)
    '''
    parent1 and parent2 are 1D np.array
    '''
    rng = np.random.default_rng(seed=seed)

    cutoff_1, cutoff_2 = np.sort(rng.choice(np.arange(len(parent1)+1), size=2, replace=False))

    def PMX_one_offspring(p1, p2):
        offspring = np.zeros(len(p1), dtype=p1.dtype)

        # Copy the mapping section (middle) from parent1
        offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

        # copy the rest from parent2 (provided it's not already there
        for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
            candidate = p2[i]
            while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                # print(f"Candidate {candidate} not valid in position {i}") # DEBUGONLY
                candidate = p2[np.where(p1 == candidate)[0][0]]
            offspring[i] = candidate
        return offspring

    offspring1 = PMX_one_offspring(parent1, parent2)
    offspring2 = PMX_one_offspring(parent2, parent1)

    return offspring1, offspring2
parent1 = np.random.permutation(10)
parent2 = np.random.permutation(10)
# Print the parents
print("Parent 1:", parent1)
print("Parent 2:", parent2)
# Apply PMX and print the offspring
offspring1, offspring2 = PMX_crossover(parent1, parent2,313)
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Parent 1: [1 3 8 7 4 6 9 2 5 0]
Parent 2: [8 3 6 4 0 7 1 9 2 5]
Offspring 1: [8 3 0 7 4 6 1 9 2 5]
Offspring 2: [1 3 8 4 0 7 9 2 5 6]


In [18]:
offspring1, offspring2 = PMX_crossover(population[0], population[1],313)
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Offspring 1: [38 41 97 11 68 44 13 99  2  0 89 24 86 94 98 53  6 62 26 67 15 45  8 90
 61 60 59 39 50 35 76 87 81 16 82 28 77 79 65 78  5 74  7 54 64 40 93 12
 63 84 33 70 20 56 51 21 30 31 47 52 96  3 71 18 58 49 69  9 37 22 73 14
 55 83 80 32 27 91 48 43 36 25  1 42 23 92 95 10 75 19 17  4 88 66 34 29
 57 46 85 72]
Offspring 2: [20 92 80  1 61 65 24 17 31 62 27 81 67 89  3 29 16 49 77 21 56  6 22 99
 64 78 50 12 39 93 15  8 42 84  7 87 95 96 51 70 36 23 60 57 69 44 53 40
 13 14 54 91  9 25 19 90 88 58 30 11 55 48 63 38 33 74 86 72 82 32  0 75
 45 28 35 43 37 83 47 79 34 41 66 26 73 85  5 59  4 52 76 97  2 46 71 94
 98 68 18 10]


In [19]:
def cxPartialyMatched(ind1, ind2):
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    return ind1, ind2
offspring1, offspring2 = cxPartialyMatched(population[0], population[1])
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Offspring 1: [ 9 92 80  1 61 19 24 17 58 49 27 42 90 94 98 93  6 62 26 67 15 45 28 99
 69 91 50 44 39 53 25 87 81 16 82 22 77 79 65 78  5 74  7 54 64 40 29 12
 63 84 33 70 20 56 51 21 30 31 47 52 96  3 71 38 57 23 86 72 60 32  0 75
 14  8 35 43 37 83 88 55 34 41 66 95 73 85 36 59  4 11 76 97  2 46 13 89
 48 68 18 10]
Offspring 2: [38 41 97 52 68 12 71 99  2  0 94 24 86 89 48 29 45 49 95 90 25 14 22 21
 61 82 59 39 50 35 76  8 42 84  7 87 26 96 51 70 36 23 60 57 69 44 53 40
 13  6 54 91  9 15 19 67 88 58 30 11 55 98 63 18 31 62 64 20 37 28 73 16
 79 83 80 32 27 78  3 43  5 56  1 81 74 92 77 10 75 65 17  4 47 66 34 93
 33 46 85 72]


In [61]:
# def erx_crossover(parent1, parent2):
#     """Performs Edge Recombination Crossover (ERX) on two chromosomes.

#     Args:
#         parent1: The first parent chromosome.
#         parent2: The second parent chromosome.

#     Returns:
#         A new chromosome resulting from the crossover.
#     """

#     offspring = []
#     visited = set()

#     # Start from a random gene in parent1
#     gene = random.choice(parent1)
#     offspring.append(gene)
#     visited.add(gene)

#     while len(offspring) < len(parent1):
#         # Find the next gene to add
#         next_gene = None

#         # Check if the next gene in parent2 is not in the visited set
#         next_gene_in_parent2 = parent2[(parent1.index(gene) + 1) % len(parent1)]
#         if next_gene_in_parent2 not in visited:
#             next_gene = next_gene_in_parent2

#         # Check if the next gene in parent1 is not in the visited set
#         if next_gene is None:
#             next_gene_in_parent1 = parent1[(parent2.index(gene) + 1) % len(parent2)]
#             if next_gene_in_parent1 not in visited:
#                 next_gene = next_gene_in_parent1

#         # If no valid next gene found, break
#         if next_gene is None:
#             break

#         # Add the next gene to the offspring and mark it as visited
#         offspring.append(next_gene)
#         visited.add(next_gene)

#         # Update the current gene
#         gene = next_gene

#     return offspring

In [69]:
import numpy as np
import random

def erx_crossover(parent1, parent2):
    """Performs Edge Recombination Crossover (ERX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    offspring = []
    visited = set()

    # Convert the parents to numpy arrays
    parent1 = np.array(parent1)
    parent2 = np.array(parent2)

    # Start from a random gene in parent1
    gene = random.choice(parent1)
    offspring.append(gene)
    visited.add(gene)

    while len(offspring) < len(parent1):
        # Find the next gene to add
        next_gene = None

        # Check if the next gene in parent2 is not in the visited set
        next_gene_in_parent2 = parent2[(np.where(parent1 == gene)[0][0] + 1) % len(parent1)]
        if next_gene_in_parent2 not in visited:
            next_gene = next_gene_in_parent2

        # Check if the next gene in parent1 is not in the visited set
        if next_gene is None:
            next_gene_in_parent1 = parent1[(np.where(parent2 == gene)[0][0] + 1) % len(parent2)]
            if next_gene_in_parent1 not in visited:
                next_gene = next_gene_in_parent1

        # If no valid next gene found, choose a random gene from the remaining ones
        if next_gene is None:
            remaining_genes = np.setdiff1d(parent1, offspring)
            next_gene = random.choice(remaining_genes)

        # Add the next gene to the offspring and mark it as visited
        offspring.append(next_gene)
        visited.add(next_gene)

        # Update the current gene
        gene = next_gene

    return offspring


In [68]:
offspring2 = erx_crossover(population[0], population[1])
# print("Offspring 1:", offspring1)
print("Offspring 2:", (offspring2))

Offspring 2: [87, 42, 86, 20, 15, 14, 83, 3, 63, 6, 49, 94, 48, 46, 34, 56, 19, 71, 18, 72, 37, 78, 36, 10, 38, 31, 30, 58, 0, 16, 7, 57, 62, 95, 74, 60, 28, 21, 88, 43, 27, 24, 99, 61, 12, 13, 93, 45, 22, 26, 90, 89, 33, 91, 59, 75, 79, 51, 67, 25, 8, 80, 52, 55, 5, 23, 64, 44, 50, 39, 35, 32, 73, 92, 97, 47, 11, 17, 2, 66, 81, 84, 54, 69, 82, 41, 1, 68, 85, 77, 96, 98, 29, 40, 53, 76, 4, 65, 70, 9]


In [81]:
import numpy as np
import random

def order_crossover(parent1, parent2):
    """Performs Order Crossover (OX1) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    offspring = np.empty_like(parent1)

    # Select a random swath of consecutive alleles from parent 1
    start = random.randint(0, len(parent1) - 1)
    end = random.randint(start + 1, len(parent1))
    swath = parent1[start:end]

    # Copy the swath to the offspring
    offspring[start:end] = swath

    # Mark out these alleles in parent 2
    marked = np.isin(parent2, swath)

    # Starting from the right side of the swath, grab alleles from parent 2 and insert them in the offspring
    index = end % len(parent1)
    for i in range(len(parent1)):
        if not marked[i]:
            offspring[index] = parent2[i]
            index = (index + 1) % len(parent1)

    return offspring
offspring2 = order_crossover(population[0], population[1])
# print("Offspring 1:", offspring1)
print("Offspring 2:", (offspring2))


Offspring 2: [62 20 37 28 73 83 80 32 27  3 43 56  1 92 10 75 17  4 47 66 34 93 33 46
 85 72 50 44 39 53 25 87 81 16 82 22 77 79 65 78  5 74  7 54 64 40 29 38
 41 97 52 68 12 71 99  2  0 94 24 86 89 48 45 49 95 90 14 21 61 59 35 76
  8 42 84 26 96 51 70 36 23 60 57 69 13  6 91  9 15 19 67 88 58 30 11 55
 98 63 18 31]


In [101]:
import numpy as np

def cycle_crossover(parent1, parent2):
    """
    Perform cycle crossover on two parents.
    """

    # Initialize child with -1s
    child = np.full_like(parent1, -1)

    # Select a random starting index
    idx = np.random.randint(0, len(parent1))

    # Perform cycle crossover
    while child[idx] == -1:
        child[idx] = parent1[idx]
        idx = np.where(parent2 == parent1[idx])[0][0]

    # Fill in the remaining values
    for i in range(len(child)):
        if child[i] == -1:
            child[i] = parent2[i]

    return child.tolist()
offspring2 = cycle_crossover(population[0], population[1])
# print("Offspring 1:", offspring1)
print("Offspring 2:", len(offspring2))


Offspring 2: 100


In [97]:
def ox_crossover(parent1, parent2):
    """Performs Order Crossover (OX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    crossover_points = random.sample(range(1, len(parent1)), 2)
    slice_start, slice_end = crossover_points[0], crossover_points[1]

    slice1 = parent1[slice_start:slice_end]
    remaining_genes1 = set(parent1) - set(slice1)

    mapping = {}
    for i in range(slice_start, slice_end):
        gene1 = parent1[i]
        gene2 = parent2[i]

        if gene1 in remaining_genes1:
            mapping[gene1] = gene2
            remaining_genes1.remove(gene1)

    offspring = []
    for gene in parent2:
        if gene in mapping:
            offspring.append(mapping[gene])
        elif gene not in slice1:
            offspring.append(gene)

    for gene in slice1:
        offspring.append(gene)

    return offspring

In [20]:
def cycle_crossover(parent1, parent2):
    """
    Performs Cycle Crossover (CX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    cycle_start = random.randrange(len(parent1))
    cycle_genes1 = set(parent1[cycle_start:])
    cycle_genes2 = set(parent2[cycle_start:])

    offspring = list(parent1[:cycle_start])
    for gene in parent2:
        if gene in cycle_genes1:
            offspring.append(gene)
            cycle_genes1.remove(gene)

            if not cycle_genes1:
                break
        else:
            cycle_genes2.add(gene)

    for gene in cycle_genes2:
        offspring.append(gene)

    return offspring

In [21]:
def swap_mutation(chromosome):
    i, j = random.sample(range(len(chromosome)), 2)
    chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
    return chromosome

In [22]:
original_chromosome = [1, 2, 3, 4, 5]
swap_mutation(original_chromosome)

[1, 2, 5, 4, 3]

In [23]:
def scramble_mutation(chromosome):
    mutation_length = random.randrange(2, len(chromosome))
    start_index = random.randrange(len(chromosome) - mutation_length)

    sub_chromosome = chromosome[start_index:start_index + mutation_length]
    random.shuffle(sub_chromosome)
    chromosome[start_index:start_index + mutation_length] = sub_chromosome

    return chromosome

In [24]:
scramble_mutation([2,1,5,4,3,8,6,7,9])

[2, 1, 5, 4, 3, 8, 6, 7, 9]

In [25]:
# def insert_mutation(chromosome):
#     i = random.randrange(len(chromosome))
#     j = random.randrange(len(chromosome))

#     if i != j:
#         gene = chromosome.pop(i)
#         chromosome.insert(j, gene)

#     return chromosome
def insert_mutation(chromosome):
    i, j = np.random.choice(len(chromosome), 2, replace=False)
    chromosome[j], chromosome[i] = chromosome[i], chromosome[j]
    return chromosome


In [26]:
insert_mutation([1,-2,-3,4,5,-6,7,8,9])

[1, -6, -3, 4, 5, -2, 7, 8, 9]

In [27]:
def inversion_mutation(chromosome):
    """Performs Inversion mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    mutation_length = random.randrange(2, len(chromosome))
    start_index = random.randrange(len(chromosome) - mutation_length)

    sub_chromosome = chromosome[start_index:start_index + mutation_length]
    sub_chromosome.reverse()
    chromosome[start_index:start_index + mutation_length] = sub_chromosome

    return chromosome
def inversion_mutation(permutation):
  # randomly select two positions in the permutation
  i, j = np.random.choice(len(permutation), size=2, replace=False)
  # make sure i is smaller than j
  if i > j:
    i, j = j, i
  # invert the segment between i and j
  permutation[i:j+1] = permutation[i:j+1][::-1]
  return permutation

In [30]:
def inversion_mutation(chromosome):
  """Performs inversion mutation on a permutation chromosome.

  Args:
    chromosome: A NumPy array of city indices in permutation representation.

  Returns:
    A NumPy array of city indices in permutation representation representing the mutated chromosome.
  """

  # Select two random indices in the chromosome.
  index1 = np.random.randint(0, len(chromosome))
  index2 = np.random.randint(0, len(chromosome))

  # Ensure that the two indices are different.
  while index1 == index2:
    index2 = np.random.randint(0, len(chromosome))

  # Reverse the order of the genes between the two indices.
  # print(type(chromosome[index1:index2 + 1]))
  chromosome[index1:index2 + 1] = chromosome[index1:index2 + 1][::-1]

  return chromosome


# Fake data
chromosome = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Mutate the chromosome
mutated_chromosome = inversion_mutation(chromosome)

# Print the mutated chromosome
print(mutated_chromosome)


[ 1  2  5  4  3  6  7  8  9 10]


In [31]:
def compare_max_values(array1, array2):
  """
  check if first array has bigger max   
  """
  min_value1 = np.min(array1)
  min_value2 = np.min(array2)

  if min_value1 < min_value2:
    return False
  else:
    return True

# Example usage
array1 = np.array([1, 2, 3, 4, 5])
array2 = np.array([6, 7, 8, 9, 10])

compare_max_values(array1, array2)

False

In [102]:
def genetic_algorithm(population_size=500, crossover_rate=0.7, mutation_rate=0.1, max_generations=700,cities = city_data):
    chromosomes = generate_random_chromosomes(500)
    print(len(chromosomes))
    fitness_values = []
    for chromosome in chromosomes :
        fitness_value = calculate_fitness_np(chromosome, cities)
        fitness_values.append(fitness_value)

    generation_without_improvement = 0

    for generation in range(max_generations):
        # Select parents using roulette wheel selection
        parents = []
        for i in range(population_size):
            parent1 , parent2 = permutation_parent_selection(chromosomes, fitness_values)
            parents.append((parent1, parent2))

        offsprings = []
        for parent1, parent2 in parents:
            # print(parent1)
            # break
            if random.random() < crossover_rate:
                offspring1 = cycle_crossover(parent1, parent2)
                offspring2 = cycle_crossover(parent1, parent2)
            else:
                offspring1 = parent1
                offspring2 = parent2

            offsprings.append(offspring1)
            offsprings.append(offspring2)

        # Apply mutation
        for offspring in offsprings:
            if random.random() < mutation_rate:
                offspring = insert_mutation(offspring)

        # Evaluate offspring fitness
        offspring_fitness_values = []
        for offspring in offsprings :
            # calculate_fitness(offspring, cities)
            offspring_fitness_value = calculate_fitness_np(offspring)
            # if offspring_fitness_value < 21285 :# todo remove this if for times reducing
            #     print(offspring_fitness_value)
            offspring_fitness_values.append(offspring_fitness_value)

        # Combine parents and offspring to form a new population
        new_population = np.empty_like(chromosomes)
        new_fitness_values = np.empty_like(fitness_values)

        for i in range(population_size):
            if offspring_fitness_values[i] < fitness_values[i]:
                new_population[i] = offsprings[i]
                new_fitness_values[i] = offspring_fitness_values[i]
            else:
                new_population[i] = chromosomes[i]
                new_fitness_values[i] = fitness_values[i]

        chromosomes = new_population
        fitness_values = new_fitness_values

        # Check for stopping condition
        if compare_max_values(offspring_fitness_values,fitness_values):
            generation_without_improvement += 1

            if generation_without_improvement >= 20:
                break
        else:
            generation_without_improvement = 0

    best_chromosome = chromosomes[fitness_values.argmax()]
    best_fitness = np.min(fitness_values)

    return best_chromosome, best_fitness


In [108]:
result = genetic_algorithm()

500


In [107]:
print(result)

(array([61, 31, 42, 80, 56, 51, 81, 69, 13, 33, 95,  2, 89,  8, 79, 88, 22,
       26, 70, 16, 94,  6,  1, 12, 55, 85, 76,  5, 37, 96, 60, 45, 98, 58,
       28, 47, 49, 75, 19, 65, 66,  4, 97, 21, 59, 32, 91, 77, 62, 20, 86,
        3,  0, 41, 43, 23, 11, 15, 10, 90, 46, 48, 30, 93, 72, 36, 38, 99,
       24, 63, 92, 67, 25, 68, 82, 64, 40, 54, 87, 57,  9, 50, 44, 73, 53,
       18, 84, 74, 17, 35, 27, 83, 39, 78, 52, 71, 14, 29, 34,  7]), 58826)
